In [1]:
import pygame
from multiprocessing import Process, Pipe

joysticks = []
pygame.init()
joysticks = [pygame.joystick.Joystick(x).get_name() for x in range(pygame.joystick.get_count())]
while joysticks == []:
    pygame.quit()
    pygame.init()
    joysticks = [pygame.joystick.Joystick(x).get_name() for x in range(pygame.joystick.get_count())]
print(joysticks)

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
['Logitech Extreme 3D']


In [2]:
if(type({}) == dict):
    print("chuj")

chuj


In [2]:
tmp = { 'h' : 1, 'm' : 2}

for k,v in tmp.items():
    print([[k], [v]])

[['h'], [1]]
[['m'], [2]]


In [44]:
from struct import pack, unpack

class Packet:
    def __init__(self, message_type: int, message: str) -> None:
        self.start = 0x69
        self.message_type = message_type
        self.message_lengh = len(message) % 256
        self.crc = (sum([ord(c) for c in message]) + len(message)) % 256
        self.message = message

    def get_packet(self) -> bytearray:
        header = pack(
            f"BBBB",
            self.start,
            self.message_type,
            self.message_lengh,
            self.crc,
        )
        return header + self.message.encode("ascii")

class dePacket:
    def __init__(self, callback):
        self.start = 0
        self.message_type = 0
        self.message_lenght = 0
        self.crc = 0
        self.payload = []
        self.deserializer_state = 0
        self.callback = callback
    # waiting_for_start = 0,
    #     waiting_for_type = 1,
    #     waiting_for_len = 2,
    #     waiting_for_crc = 3,
    #     data_acquisition = 4
    def deserialize(self, data):
        for i in range(len(data)):  
            s = self.deserializer_state
            tmp = data[i]
            packet_start = 0x69

            # case waiting_for_start:
            if(s == 0):
                if(tmp == packet_start):
                    self.start = tmp
                    self.payload = []
                    self.deserializer_state = 1
                    
            
            # case waiting_for_type:
            elif(s == 1):
                self.message_type = tmp
                self.deserializer_state = 2
                

            # case waiting_for_len:
            elif(s == 2):
                self.message_lenght = tmp
                self.deserializer_state = 3
                
            
            # case waiting_for_crc:
            elif(s == 3):
                self.crc = tmp
                self.deserializer_state = 4
                
            # case data_acquisition:
            elif(s == 4):
                self.payload.append(tmp)

            print(f'{len(self.payload)}, {self.message_lenght}, {self.deserializer_state }, {tmp}')
            if(len(self.payload) == self.message_lenght and self.deserializer_state == 4):
                tmp_crc = (sum([ord(c) for c in str(bytes(self.payload), 'utf-8')]) + len(self.payload)) % 256

                if(tmp_crc == self.crc):
                    self.callback(self.message_type, self.payload)
                
                if(i > 0): 
                    i -= 1
                self.deserializer_state = 0

def callback(type, payload):
    print(f"Received a packet. Type: {type}, Payload: {payload}")

In [48]:
tmp = Packet(1, '')
print(tmp)
deserializer = dePacket(callback)

p = tmp.get_packet()

deserializer.deserialize(p[0:1])


0, 0, 1, 105
0, 0, 2, 1
0, 0, 3, 0
0, 0, 4, 0
Received a packet. Type: 1, Payload: []
